In [4]:
%load_ext autoreload
%autoreload 2
import os
import gym

from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv

# env = DummyVecEnv([lambda: gym.make("Watercup-v0")])
env = make_vec_env("Watercup-v0", n_envs=1)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="./ppo_tensorboard/")

timestep = 3*5e5
model.learn(total_timesteps=timestep, tb_log_name="run", progress_bar=True)

log_dir = "./trained_policy/"
model.save(log_dir + "ppo_watercup")
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env.save(stats_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using cpu device
Logging to ./ppo_tensorboard/run_2


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 3.35     |
|    ep_rew_mean     | 13.7     |
| time/              |          |
|    fps             | 58       |
|    iterations      | 1        |
|    time_elapsed    | 35       |
|    total_timesteps | 2048     |
---------------------------------


In [1]:
# evaluate after training
%load_ext autoreload
%autoreload 2
import os
import gym
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv

log_dir = "./trained_policy/"
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env = make_vec_env("Watercup-v0", n_envs=1)
env = VecNormalize.load(stats_path, env)
env.training = False
env.norm_reward = False

model = PPO.load(log_dir + "ppo_watercup", env=env)

mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10, render=False)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

objc[17532]: Class GLFWApplicationDelegate is implemented in both /Users/tychen/.mujoco/mujoco210/bin/libglfw.3.dylib (0x1199f0778) and /usr/local/Cellar/glfw/3.3.8/lib/libglfw.3.3.dylib (0x11f555790). One of the two will be used. Which one is undefined.
objc[17532]: Class GLFWWindowDelegate is implemented in both /Users/tychen/.mujoco/mujoco210/bin/libglfw.3.dylib (0x1199f0700) and /usr/local/Cellar/glfw/3.3.8/lib/libglfw.3.3.dylib (0x11f5557b8). One of the two will be used. Which one is undefined.
objc[17532]: Class GLFWContentView is implemented in both /Users/tychen/.mujoco/mujoco210/bin/libglfw.3.dylib (0x1199f07a0) and /usr/local/Cellar/glfw/3.3.8/lib/libglfw.3.3.dylib (0x11f555808). One of the two will be used. Which one is undefined.
objc[17532]: Class GLFWWindow is implemented in both /Users/tychen/.mujoco/mujoco210/bin/libglfw.3.dylib (0x1199f0818) and /usr/local/Cellar/glfw/3.3.8/lib/libglfw.3.3.dylib (0x11f555880). One of the two will be used. Which one is undefined.
/opt/a

Mean reward = 142.58 +/- 0.00


In [ ]:
# visualize trained policy
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv

log_dir = "./trained_policy/"
stats_path = os.path.join(log_dir, "vec_normalize.pkl")
env = make_vec_env("Watercup-v0", n_envs=1)
env = VecNormalize.load(stats_path, env)
env.training = False
env.norm_reward = False

model = PPO.load(log_dir + "ppo_watercup", env=env)

t = 0
horizon = 1000
obs = env.reset()
# print("initial distance: ", np.linalg.norm(obs[-3:]))
rewards = []
done = False
# action = [0.174, 0.0793, 1.05, 1.22, 0.209, 0.698, 0.81, -0.119, 1.04, 1.29, -0.164, 1.14, 1.29, 0.119, 1.17, 1.23, 0.475, 0.115, 1.2, 1.24, 0.0525, 0, -0.027+0.07, 0, -0.06, 0]
while t < horizon and not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = env.step(action)
    rewards.append(reward)
    # print("info: ", info["init_dist"], info["curr_dist1"], info["curr_dist2"])
    # print("action: ", info["action"])
    # print("nstep: ", info["nstep"])
    # print("not_in_hand: ", done)
    # print("sensor: ", info["sensor"])
    # print("in_hand_reward: ", info["reward_in_hand"])
    env.render()
    t += 1
env.close()
print(len(rewards))
print(sum(rewards))